In [1]:
%pip install requests
%pip install xmltodict


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import requests
from datetime import datetime
import os
import gzip
import shutil

from epgs import urls

xmls = []

def obtener_extension(filename):
    return os.path.splitext(filename)[1]

# Ruta de la carpeta y el archivo
folder_path = 'downloads'

# Obtener la fecha y hora actual
fechahoraactual = datetime.now().strftime("%Y%m%d%H%M%S")

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Carpeta '{folder_path}' creada.")

for name, url in urls:
    extension = obtener_extension(name)
    filename = f"{folder_path}/{name}"

    response = requests.get(url)
    with open(filename, 'wb') as file:
        file.write(response.content)

    print(f"Archivo guardado como {filename}")

    if extension == '.gz':
        uncompressed_filename = filename[:-3]

        with gzip.open(filename, 'rb') as f_in:
            with open(uncompressed_filename, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

        print(f"Archivo descomprimido como {uncompressed_filename}")
        xmls.append(uncompressed_filename)


folder_path = 'downloads'
from canales import incluir

xmls = [
    "downloads/epg_ripper_MX1.xml",
    "downloads/jmQcr6fwJB.xml"
]

canales = []
programas = []

for filename in xmls:


    with open(filename, 'r', encoding='utf-8') as file:
        data = file.read()

        import xmltodict

        xml_dict = xmltodict.parse(data)

        for i in incluir:
            canales += [p for p in xml_dict["tv"]["channel"] if p["@id"] == i]
            programas += [p for p in xml_dict["tv"]["programme"] if p["@channel"] == i]

        programacion = {
            'tv': {
                '@generator-info-name': 'none',
                '@generator-info-url': 'none',
                'channel': canales,
                'programme': programas
            }
        }

xmltodict.unparse(programacion, output=open('epg.xml', 'w', encoding="utf-8"), pretty=True)


Carpeta 'downloads' creada.
Archivo guardado como downloads/epg_ripper_MX1.xml.gz
Archivo descomprimido como downloads/epg_ripper_MX1.xml
Archivo guardado como downloads/jmQcr6fwJB.xml
